In [1]:
!git clone https://github.com/zayservl/AI_innopolis_arch.git

Cloning into 'AI_innopolis_arch'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 45 (delta 13), reused 35 (delta 6), pack-reused 0
Receiving objects: 100% (45/45), 28.28 MiB | 10.74 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
#Imports
! pip install transformers
import string
import spacy
import torch
import nltk

import torch.nn as nn
import pandas as pd
import numpy as np


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from transformers import AutoTokenizer,\
DistilBertForSequenceClassification, DistilBertModel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.7 MB/s eta 0:00:00


In [4]:
#Модель лемматизации
nlp = spacy.load("en_core_web_sm")

#Загрузка стоп-слов и пунктуации
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
punctuation = string.punctuation

#preprocessing
def preprocess_and_lemmatize(text):
    lemma_text = nlp(" ".join(text.lower().split()))
    lemmatized_words = [
        token.lemma_ for token in lemma_text
        if token.text not in punctuation and token.text not in stop_words
        ]
    return " ".join(lemmatized_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Задание №1

In [5]:
with open('/content/AI_innopolis_arch/10_songs.txt') as fr:
  all_songs = fr.read().split('!splitter!')

In [6]:
preprocessed_songs = [preprocess_and_lemmatize(song) for song in all_songs]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_songs)
feature_names = tfidf_vectorizer.get_feature_names_out()
for idx, score in enumerate(tfidf_matrix.toarray()):
  # Сортировка слов по значениям TF-IDF
  sorted_keywords = [word for _, word in sorted(zip(score, feature_names), reverse=True)]

  print(f"Ключевые слова для песни # {idx+1}: {sorted_keywords[:5]}")

Ключевые слова для песни # 1: ['be', 'slim', 'shady', 'please', 'stand']
Ключевые слова для песни # 2: ['will', 'spot', 'whoa', 'shop', 'candy']
Ключевые слова для песни # 3: ['sha', 'ba', 'da', 'good', 'ha']
Ключевые слова для песни # 4: ['go', 'let', 'mama', 'really', 'matter']
Ключевые слова для песни # 5: ['rollin', 'hand', 'keep', 'back', 'move']
Ключевые слова для песни # 6: ['be', 'toxic', 'taste', 'not', 'slippin']
Ключевые слова для песни # 7: ['without', 'inside', 'wake', 'save', 'bring']
Ключевые слова для песни # 8: ['become', 'undertow', 'catch', 'tired', 've']
Ключевые слова для песни # 9: ['singe', 'ohh', 'voice', 'million', 'oh']
Ключевые слова для песни # 10: ['black', 'paint', 'turn', 'see', 'red']


Задание №2

In [7]:
N_EPOCH = 20
MODEL_NAME = 'distilbert-base-uncased'#"bert-base-uncased"
BATCH_SIZE = 16
N_DATA_SELECTION = 1000
TYPE_OF_MODEL = 'user' #'bert'

In [8]:
# Load the BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)#DistilBertTokenizer.from_pretrained(model_name)#BertTokenizer.from_pretrained(model_name)
distil_model_clf = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
#Configurate our model
#В BErt еще используется pre-clasifier - добавим тоже
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert = DistilBertModel.from_pretrained(MODEL_NAME)
        self.pre_classifier = torch.nn.Linear(
            self.bert.config.hidden_size, self.bert.config.hidden_size
          )
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, 2)
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, ids, attention_mask):
        bert_output = self.bert(
            ids,
            attention_mask = attention_mask,
            return_dict=False
            )[0][:, 0]
        pre_clf_output = self.pre_classifier(bert_output)
        relu_out = torch.nn.ReLU()(pre_clf_output)
        drop_out = self.dropout(relu_out)
        class_out = self.classifier(drop_out)

        return class_out

if TYPE_OF_MODEL == 'user':
  model = BERTClass()
else:
  model = distil_model_clf

Подготовка данных

In [10]:
 #Скольк овозьмем данных из дадасета
df_imdb = pd.read_csv('/content/AI_innopolis_arch/IMDB Dataset.csv')
df_imdb = pd.read_csv('/content/AI_innopolis_arch/IMDB Dataset.csv')
df_imdb.columns = ['text', 'label']
df_imdb['label'] = df_imdb['label'].apply(lambda x: 0 if x == 'positive' else 1)
#Все не влазит в память
df_imdb = df_imdb.loc[:N_DATA_SELECTION]

#Подготовим данные
label_data = df_imdb['label'].tolist()
text_data = [preprocess_and_lemmatize(_) for _ in df_imdb['text'].tolist()]

In [12]:
x_train, x_test, y_train, y_test = train_test_split(
    text_data,
    label_data,
    test_size=0.3,
    random_state = 42)

In [13]:
# Токенизация и кодирование данных
def tokenize(array_of_sent, _tokenizer):
  return _tokenizer(array_of_sent, max_length=512,
                   truncation=True, padding='max_length',
                   add_special_tokens=True, return_attention_mask=True,
                   return_token_type_ids=False, return_tensors='pt'
                   )

train_encodings = tokenize(x_train, tokenizer)
test_encodings = tokenize(x_test, tokenizer)

In [14]:
def prepare_datase_pytorch(ecnoding_data, labels):
  return torch.utils.data.TensorDataset(
    ecnoding_data['input_ids'],
    ecnoding_data['attention_mask'],
    torch.tensor(labels)
  )

train_dataset = prepare_datase_pytorch(train_encodings, y_train)
test_dataset = prepare_datase_pytorch(test_encodings, y_test)

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True
                                           )

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model.to(device)

cuda


BERTClass(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_fea

In [17]:
for epoch in range(N_EPOCH):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids ,attention_mask, target  = [_.to(device) for _ in batch]
        optimizer.zero_grad()

        output = model(input_ids, attention_mask=attention_mask)
        output = output[0] if TYPE_OF_MODEL == 'bert' else output

        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

Epoch 1, Loss: 30.181989133358
Epoch 2, Loss: 25.507806926965714
Epoch 3, Loss: 15.247194737195969
Epoch 4, Loss: 11.439057528972626
Epoch 5, Loss: 5.473664149641991
Epoch 6, Loss: 2.6295286593958735
Epoch 7, Loss: 2.37547311373055
Epoch 8, Loss: 1.0798240043222904
Epoch 9, Loss: 0.8140362706035376
Epoch 10, Loss: 0.38297973247244954
Epoch 11, Loss: 0.22993960534222424
Epoch 12, Loss: 0.17897195369005203
Epoch 13, Loss: 0.14831516635604203
Epoch 14, Loss: 1.4191647910047323
Epoch 15, Loss: 5.032484381459653
Epoch 16, Loss: 0.9742198006715626
Epoch 17, Loss: 0.3517923953477293
Epoch 18, Loss: 0.17786509287543595
Epoch 19, Loss: 0.11150107393041253
Epoch 20, Loss: 0.09657403209712356


In [21]:
# Оценка модели
model.eval()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, _  = [_.to(device) for _ in batch]

        output = model(input_ids, attention_mask=attention_mask)
        output = output[0] if TYPE_OF_MODEL == 'bert' else output

        predicted_labels = torch.argmax(output, dim=1).tolist()
        predictions.extend(predicted_labels)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8305647840531561


In [28]:
def test(test_sent, model, device, tokenizer):
  train__sent_encodings = tokenize([test_sent], tokenizer)
  out = model(
      train__sent_encodings['input_ids'].to(device),
      train__sent_encodings['attention_mask'].to(device)
  )
  predicted_label = torch.argmax(out, dim=1).tolist()[0]
  return 'positive' if predicted_label == 0 else 'negative'

In [32]:
list_of_test = ['I hate this film!',
                'I love this film!',
                'I recommend this film to those who love fantasy and adventures.',
                'This film like dog shit'
                ]

[print(f"{_}: {test(_,  model, device, tokenizer)}") for _ in list_of_test]

I hate this film!: negative
I love this film!: positive
I recommend this film to those who love fantasy and adventures.: positive
This film like dog shit: negative


[None, None, None, None]